In [1]:
import os
import time
import numpy as np
from datetime import timedelta
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from kmeans_pytorch import kmeans

c:\Users\user\Miniconda3\envs\rec\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Data Load and Dataset

학습된 item embedding이 필요하므로 공식 github에 있는 dataset과 item embedding 사용.

In [2]:
def data_load(dir_path):
    train_path = os.path.join(dir_path, 'train_list.npy')
    valid_path = os.path.join(dir_path, 'valid_list.npy')
    test_path = os.path.join(dir_path, 'test_list.npy')
    emb_path = os.path.join(dir_path, 'item_emb.npy')

    train_list = np.load(train_path, allow_pickle=True)
    valid_list = np.load(valid_path, allow_pickle=True)
    test_list = np.load(test_path, allow_pickle=True)

    uid_max = 0
    iid_max = 0
    train_dict = {}

    for uid, iid in train_list:
        if uid not in train_dict:
            train_dict[uid] = []
        train_dict[uid].append(iid)
        if uid > uid_max:
            uid_max = uid
        if iid > iid_max:
            iid_max = iid
    
    n_user = uid_max + 1
    n_item = iid_max + 1
    print(f'user num: {n_user}')
    print(f'item num: {n_item}')

    train_data = sp.csr_matrix((np.ones_like(train_list[:, 0]), \
        (train_list[:, 0], train_list[:, 1])), dtype='float64', \
        shape=(n_user, n_item))
    
    valid_y_data = sp.csr_matrix((np.ones_like(valid_list[:, 0]),
                 (valid_list[:, 0], valid_list[:, 1])), dtype='float64',
                 shape=(n_user, n_item))  # valid_groundtruth

    test_y_data = sp.csr_matrix((np.ones_like(test_list[:, 0]),
                 (test_list[:, 0], test_list[:, 1])), dtype='float64',
                 shape=(n_user, n_item))  # test_groundtruth
    

    emb_items = torch.from_numpy(np.load(emb_path, allow_pickle=True))
    
    return train_data, valid_y_data, test_y_data, n_user, n_item, emb_items

In [3]:
class DataDiffusion(Dataset):
    def __init__(self, dataset):
        self.data = dataset

    def __getitem__(self, index): 
        item = self.data[index]
        return item
        
    def __len__(self):
        return len(self.data)

### 2. Diffusion and Model

#### 2.1. Diffusion

In [4]:
class Diffusion():
    def __init__(self, steps=100, beta_start=1e-4, beta_end=0.02, device='cuda',\
            noise_scale=0.1, num_for_expectation=10):
        """
        Forward diffusion 또는 주어진 model로 reverse diffusion한다.
        Args:
            steps               : reverse할 개수
            beta_start          : Beta 시작 값, DDPM 논문에 적힌 1e-4 사용.
            beta_end            : Beta 마지막 값, DDPM 논문에 적힌 0.02 사용. 
            noise_scale         : Beta를 생성할 때, noise의 정도를 조정하기 위해 사용.
                                : 논문 3.4 personalized recommendation 1)의 마지막 문장 참고.
            num_for_expectation : Importance sampling에 사용되는, expectation을 구하기 위해 필요한 loss의 수
        """
        self.steps = steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.device = device
        self.noise_scale = noise_scale

        self.beta = torch.tensor(self.get_betas(), dtype=torch.float32).to(device)
        self.alpha = 1.0 - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
        self.alpha_bar_prev = torch.cat([torch.tensor([1.0]).to(device), self.alpha_bar[:-1]]).to(device) # 제일 처음 원소는 어차피 안 쓰임.
        self.sqrt_alpha_bar = torch.sqrt(self.alpha_bar)
        self.sqrt_one_minus_alpha_bar = torch.sqrt(1 - self.alpha_bar)

        # 논문 수식 8
        self.posterior_mean_coef1 = torch.sqrt(self.alpha) * (1 - self.alpha_bar_prev) / (1.0 - self.alpha_bar) # x_t 앞의 계수
        self.posterior_mean_coef2 = torch.sqrt(self.alpha_bar_prev) * self.beta / (1.0 - self.alpha_bar) # batch 앞의 계수
        # 아래 부분은 posterior variance를 사용하는 값인 것 같다.
        # 원래 DDPM에서는 beta를 사용했지만, 학습할 수도 있고, 아래처럼 다양한 것을 사용할 수 있다.
        # 본 논문에서는 따로 언급은 없지만 log_var_clipped를 사용했다.
        self.posterior_variance = self.beta * (1.0 - self.alpha_bar_prev) / (1.0 - self.alpha_bar)
        self.posterior_log_variance_clipped = torch.log(
            torch.cat([self.posterior_variance[1].unsqueeze(0), self.posterior_variance[1:]])
        )

        # Step을 sampling 하는 방법 중 importance sampling에 필요한 variable.
        # Importance sampling은 DDPM을 향상 시키는 technique 중 하나.
        # Importance sampling은 각 step마다 optimization의 어려움 정도가 다르다는 것을 가정한다.
        # 그래서 Loss가 큰 step에 대한 학습을 강조하기 위해 importance sampling을 고려한다.
        # 간단히 말하면 loss가 큰 step에 대해 sampling 확률을 높인다.
        self.num_for_expectation = num_for_expectation # Monte Calro를 사용하기 위한 개수.
        self.Lt_history = torch.zeros(steps, num_for_expectation, dtype=torch.float32).to(device)
        self.Lt_count = torch.zeros(steps, dtype=int).to(device)

    def get_betas(self, max_beta=0.999):
        # DDPM에서는 beta가 linear하게 증가하도록 하고 있는데,
        # 본 논문 eq 4는 alpha_bar가 linear하도록 beta를 설정하고 있다.
        # 풀어서 써보면 alpha_bar = 1 - np.linspace(...)의 값을 가지게 된다.

        start = self.noise_scale * self.beta_start
        end = self.noise_scale * self.beta_end
        alpha_bar = 1 - np.linspace(start, end, self.steps) # 1 - \beta
        betas = []
        betas.append(1 - alpha_bar[0])
        for i in range(1, self.steps):
            betas.append(min(1 - alpha_bar[i] / alpha_bar[i - 1], max_beta))
        return np.array(betas)

    def sample_steps(self, batch_size, method='uniform', uniform_prob=0.001):
        if method == 'importance':
            if not (self.Lt_count == self.num_for_expectation).all():
                # 모든 steps에 대한 loss가 num_for_expectation만큼 없으면 uniform 방식으로 sampling
                return self.sample_steps(batch_size, method='uniform')

            # 수식 14에 따라 sampling한다.
            Lt_sqrt = torch.sqrt(torch.mean(self.Lt_history ** 2, axis=-1)) 
            # 10개 loss의 제곱에 대한 평균의 루트값.
            # Lt_sqrt shape: (steps,)
            
            pt_prob = Lt_sqrt / torch.sum(Lt_sqrt)
            pt_prob *= 1 - uniform_prob 
            pt_prob += uniform_prob / len(pt_prob)
            # Loss의 크기에 따라 sampling 확률을 다르게 준다.
            # 어느 정도 uniform_prob 만큼 sampling 되는 것을 보장.

            step = torch.multinomial(input=pt_prob, num_samples=batch_size, replacement=True) 
            # 중복 sampling
            pt = pt_prob.gather(dim=0, index=step) * len(pt_prob)
            # 각 step의 확률 값을 가져온다.
            # 수식 14에 따라 training에서 Lt / pt를 하기 위함.

        elif method == 'uniform':
            steps = torch.randint(low=1, high=self.steps, size=(batch_size,)).long()
            pt = torch.ones_like(steps).float()
            # loss의 평균 값을 구하기 때문에 len(pt)로 안 나눠줘도 된다.
        
        else: raise ValueError
        
        return steps.to(self.device), pt.to(self.device)

    def get_noised_interaction(self, batch, t):
        """
        Noise를 추가한, 각 item에 대한 소비할 확률 값을 구하는 함수.
        논문 수식 3 참고.
        Args:
            batch : Training dataset으로 만들어진 user interactions   (batch_size, num_items)
            t   : noise step                                       (batch_size, )
        """
        sqrt_alpha_bar = self.sqrt_alpha_bar[t][:, None]
        mean_ = sqrt_alpha_bar * batch
        std_ = self.sqrt_one_minus_alpha_bar[t][:, None]
        noise = torch.randn_like(batch)
        # 각 item마다 줄 noise를 sampling한다. -> reparameter trick에서 사용됨.

        noised_interaction = mean_ + std_ * noise # reparmeter
        return noised_interaction, noise


    def sample_new_interaction(self, model, batch, steps: int, sampling_noise=False):
        """
        batch부터 정해진 steps만큼 noise를 주고, 
        학습된 model을 가지고 reverse diffusion으로 추천을 생성한다.
        Args:
            model           : 학습된 model
            batch             : 초기 users의 interactions, shape: (batch_size, num_items), shape (batch_size, num_items)
            steps           : Forward를 할 step, x_T까지 forward하지 않는다. 이유는 논문 3.3 참고.
            sampling_noise  : 추천을 생성할 때, noise 추가 유무. False이면 variance 없이 mean 값만 사용.
        """

        if steps == 0: 
            # noise를 전혀 추가하지 않고, reverse를 T번 진행
            # 기존의 user의 interaction이 noise하다고 가정.
            x_T = batch
        else:
            T = torch.tensor([steps - 1] * batch.shape[0]).to(batch.device)
            x_T, noise = self.get_noised_interaction(batch, T)

        reverse_t = list(range(self.steps))[::-1]
        
        x_t = x_T
        if self.noise_scale == 0:
            # Denoise 과정이 없다.
            # 즉, forward가 없다. 이러면 각 reverse는 x_t -> x_t를 복원하는 것이다.
            # 결국 x_t -> x_t를 하는 AutoEndocer를 여러 개 쌓은 것과 같다.
            for t_idx in reverse_t:
                t = torch.tensor([t_idx] * x_t.shape[0]).to(batch.device) # Shape: (batch_size, )
                x_t = model(x_t, t)
        else:
            # Denosing 과정이 있다.
            # x_t와 t가 주어졌을 때, p(x_{t-1}|x_t)의 평균과 분산 값을 구한다.
            # 이를 통해 reparameterzation trick으로 x_{t-1}을 얻는다.
            # 우리는 p(x_{t-1}|x_t)를 모른다. 여러 수식을 유도하면, p(x_{t-1}|x_t)의 likelihodd는 
            # q(x_{t-1}|x_t, batch)와 유사한 분포를 가지면 커진다.
            # 이때, x_t는 알아도 batch를 모른다. 그래서 학습된 model을 통해 batch를 예측한 값을 사용한다.
            for t_idx in reverse_t:
                t = torch.tensor([t_idx] * x_t.shape[0]).to(batch.device)
                batch_hat = model(x_t, t)
                mean_hat = self.posterior_mean_coef1[t][:, None] * x_t + self.posterior_mean_coef2[t][:, None] * batch_hat
                if sampling_noise:
                    # 추천을 생성할 때 uncertainty 추가. 즉, variance 사용
                    variance = self.posterior_log_variance_clipped[t][:, None]
                    if t_idx > 1: noise = torch.randn_like(x_t)
                    else: noise = torch.zeros_like(x_t) # t == 0일 때 noise를 주지 않는다.
                    x_t = mean_hat + torch.exp(0.5 * variance) * noise
                else:
                    # 추천을 생성할 때 variance를 사용하지 않음.
                    # 따라서 평균값만 사용.
                    x_t = mean_hat
        return x_t

#### 2.2. Model

In [5]:
class VAE(nn.Module):
    def __init__(self, num_items, emb_items, num_clusters=3, hidden_dims=[300], \
            device='cuda', dropout=0.1):
        super(VAE, self).__init__()

        self.encoder = nn.ModuleList([])
        self.decoder = nn.ModuleList([])
        self.dropout = nn.Dropout(dropout)
        self.active = nn.Tanh()
        self.num_clusters = num_clusters
        self.hidden_dims = hidden_dims

        if num_clusters == 1: 
            # no clustering
            # 굳이 if 문을 안 둬도 되지 않나??
            in_dims_tmp = [num_items] + hidden_dims[:-1] + [hidden_dims[-1] * 2]
            out_dims_tmp = in_dims_tmp[::-1]
            for d_in, d_out in zip(in_dims_tmp[:-1], in_dims_tmp[1:]):
                self.encoder.append(nn.Linear(d_in, d_out))
                self.encoder.append(self.active)
            for d_in, d_out in zip(out_dims_tmp[:-1], out_dims_tmp[1:]):
                self.decoder.append(nn.Linear(d_in, d_out))
                self.decoder.append(self.active)
            self.decoder = self.decoder[:-1]
        else:
            # Clustering
            #### Build cluster map ####
            self.cluster_ids, centers = kmeans(X=emb_items, num_clusters=num_clusters, distance='euclidean', device=device)
            # cluster_ids(labels): [0, 1, 2, 2, 1, 0, 0, ...]
            # item이 순서대로 어떤 cluster에 해당하는지 나타낸다.
            self.cluster_idx = [] 
            # cluster_idx:
            #   각 cluster 별 어떤 item이 담기는지 저장.
            #   이후 batch interaction이 들어왔을 때, cluster로 분배해주기 위해 필요하다.
            for idx in range(num_clusters):
                indicies = np.argwhere(self.cluster_ids.numpy() == idx).squeeze().tolist()
                self.cluster_idx.append(torch.tensor(indicies))
            self.cluster_map = torch.cat(tuple(self.cluster_idx), dim=0)
            # cluster_map:
            #   Evaluation에서 생성된 items들이 순서대로 어떤 item id에 대한 점수인지를 나타낼 때 사용한다.
            self.num_items_per_cluster = [len(self.cluster_idx[idx]) for idx in range(num_clusters)]
            # num_items_per_cluster:
            #   encoder, decoder dimension 계산에 사용됨.
            
            #### Build encoder and decoder ####
            self.in_latent_dims_clusters = []
            for idx in range(num_clusters):
                # Cluster에 있는 item의 수에 따라 정해진 dimension(논문에선 300을 사용)을 나눠 가진다.
                if idx == num_clusters - 1:
                    # 마지막 cluster의 dimension은 나머지 값을 가진다.
                    latent_dims = [hidden_dims[j] - np.array(self.in_latent_dims_clusters)[:,j].sum(axis=0) for j in range(len(hidden_dims))]
                else:
                    latent_dims = [int(self.num_items_per_cluster[idx] / num_items * hidden_dims[j]) \
                        for j in range(len(hidden_dims))] # Cluster에 있는 item의 수에 따라 j번째 hidden dimension을 나눈다.
                    latent_dims = [latent_dims[j] if latent_dims[j] != 0 else 1 \
                        for j in range(len(hidden_dims))] # 만약 item의 수가 너무 적어 dimension 값이 0이 나오면, 1을 준다.
                
                self.in_latent_dims_clusters.append(latent_dims) 
                # latent_dims_clusters:
                #   마지막 cluster의 dims를 구하기 위해 필요
                #   그리고 decoder in, out dim에 필요
                
                in_dims_tmp = [self.num_items_per_cluster[idx]] + latent_dims[:-1] + [latent_dims[-1] * 2]
                encoder_tmp = nn.ModuleList([])
                for d_in, d_out in zip(in_dims_tmp[:-1], in_dims_tmp[1:]):
                    encoder_tmp.append(nn.Linear(d_in, d_out))
                    encoder_tmp.append(self.active)
                self.encoder.append(nn.Sequential(*encoder_tmp))
                del encoder_tmp
            
            self.out_latent_dims_clusters = []            
            for idx in range(self.num_clusters):
                out_dim_tmp = self.in_latent_dims_clusters[idx][::-1] + [self.num_items_per_cluster[idx]]
                self.out_latent_dims_clusters.append(out_dim_tmp)
                decoder_tmp = nn.ModuleList([])
                for d_in, d_out in zip(out_dim_tmp[:-1], out_dim_tmp[1:]):
                    decoder_tmp.append(nn.Linear(d_in, d_out))
                    decoder_tmp.append(self.active)
                self.decoder.append(nn.Sequential(*decoder_tmp[:-1])) # 마지막엔 activate function 추가 안 함.
                del decoder_tmp
        
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    nn.init.normal_(m.bias.data, mean=0.0, std=0.001)

    def Encode(self, batch):
        batch = self.dropout(batch)
        if self.num_clusters == 1:
            latent = self.encoder(batch)
            mu = latent[:, :self.hidden_dims[-1]]
            log_var = latent[:, self.hidden_dims[-1]:]

            # self.training: model.train()일 때 True, model.eval()일 때 False
            if self.training and self.reparam: noise = torch.randn_like(log_var) # log_var_clipped
            else: noise = torch.zeros_like(log_var) # noise 안 줌.
            latent = mu + torch.exp(0.5 * log_var) * noise

            kl_divergence = -0.5 * torch.mean(torch.sum(1 + log_var - mu.pow(2) - log_var.exp(), dim=1))
        else:
            batch_per_cluster = []
            for idx in range(self.num_clusters):
                batch_per_cluster.append(batch[:, self.cluster_idx[idx]])
            # (batch_size, num_items) -> ((batch_size, cluster1), (batch_size, cluster2), ...)
            cluster_mu = []
            cluster_log_var = []
            for idx in range(self.num_clusters):
                cluster_latent = self.encoder[idx](batch_per_cluster[idx])
                cluster_mu.append(cluster_latent[:, :self.in_latent_dims_clusters[idx][-1]])
                cluster_log_var.append(cluster_latent[:, self.in_latent_dims_clusters[idx][-1]:])

            mu = torch.cat(tuple(cluster_mu), dim=-1)
            log_var = torch.cat(tuple(cluster_log_var), dim=-1)

            if self.training: noise = torch.randn_like(log_var) # log_var_clipped
            else: noise = torch.zeros_like(log_var) # noise 안 줌.
            latent = mu + torch.exp(0.5 * log_var) * noise

            # Multinomial likelihood KL_D -> MultVAE 참고.
            kl_divergence = -0.5 * torch.mean(torch.sum(1 + log_var - mu.pow(2) - log_var.exp(), dim=1))

            batch_cluster = torch.cat(tuple(batch_per_cluster), dim=-1)
        
        # batch는 cluster 순서에 따라 정렬된 items로, decoder와의 loss 계산에 필요
        # latent는 diffusion을 위해 필요
        # kl_divergence는 loss계산에서 annealing한다. -> MultVAE와 같다.
        return batch_cluster, latent, kl_divergence

    def Decode(self, batch_latent):
        if self.num_clusters == 1:
            batch_recon = self.decoder(batch_latent)
        else:
            batch_cluster_recon = []
            start = 0
            for idx in range(self.num_clusters):
                end = start + self.out_latent_dims_clusters[idx][0]
                batch_cluster_recon.append(self.decoder[idx](batch_latent[:, start:end]))
                start = end
            batch_recon = torch.cat(tuple(batch_cluster_recon), dim=-1)
        
        return batch_recon

In [6]:
class PosEmb(nn.Module):
    def __init__(self, pos_dim):
        super(PosEmb, self).__init__()
        """
        Sinusoidal timestep positional encoding.
        Args
            pos_dim : embedding dimension
        """
        self.pos_dim = pos_dim
        self.time_mlp = nn.Linear(pos_dim, pos_dim)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    nn.init.normal_(m.bias.data, mean=0.0, std=0.001)

    def forward(self, t, max_period=10000):
        t = t.unsqueeze(-1).type(torch.float)
        half_dim = self.pos_dim // 2
        w_k = 1.0 / (
            max_period
            ** (torch.arange(0, half_dim, 1, device=t.device).float() / (half_dim-1))
        )

        half_emb = t.repeat(1, half_dim)
        pos_sin = torch.sin(half_emb * w_k)
        pos_cos = torch.cos(half_emb * w_k)
        pos_enc = torch.cat([pos_sin, pos_cos], dim=-1)

        emb = self.time_mlp(pos_enc)
        return emb


class ReverseDiffusion(nn.Module):
    def __init__(self, hidden_dims, latent_dim, step_dim=10, norm=False, droupout=0.5):
        super(ReverseDiffusion, self).__init__()
        """
        AutoEncoder 구조를 활용한다.
        Args
            hidden_dims : MLP dims for predict batch
            latent_dim  : Latent vector dimension
            step_dim    : timestep positional embedding dim.
            norm        : batch normalization
            dropout     : dropout
        """

        self.norm = norm
        self.pos_enc_layer = PosEmb(step_dim)
        self.encoder = nn.ModuleList([])
        self.decoder = nn.ModuleList([])
        in_dims = [latent_dim] + hidden_dims
        out_dims = in_dims[::-1]
        # in_dims     : [latent_dim, ...]
        # out_dims    : [..., latent_dim]

        in_dims_w_step = [in_dims[0] + step_dim] + in_dims[1:] # [latent_dim + pos_emb_size] + hidden_dims
        for d_in, d_out in zip(in_dims_w_step[:-1], in_dims_w_step[1:]):
            self.encoder.append(nn.Linear(d_in, d_out))
            self.encoder.append(nn.Tanh())
        for d_in, d_out in zip(out_dims[:-1], out_dims[1:]):
            self.decoder.append(nn.Linear(d_in, d_out))

        self.drop = nn.Dropout(droupout)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    nn.init.normal_(m.bias.data, mean=0.0, std=0.001)

    def forward(self, x, timesteps):
        time_emb = self.pos_enc_layer(timesteps)
        if self.norm: x = F.normalize(x)
        x = self.drop(x)
        h = torch.cat([x, time_emb], dim=-1)
        for idx, layer in enumerate(self.encoder):
            h = layer(h)
        for idx, layer in enumerate(self.decoder):
            h = layer(h)
            if idx != len(self.decoder) - 1:
                h = torch.tanh(h)

        return h

### 3. Training

#### 3.1. utils

In [7]:
def SNR(diffusion, t):
    """
    Compute the signal-to-noise ratio for a single timestep.
    """
    diffusion.alpha_bar = diffusion.alpha_bar.to(t.device)
    return diffusion.alpha_bar[t] / (1 - diffusion.alpha_bar[t])


def caculate_loss(args, model_reverse, model_vae, diffusion, batch, update_count, update_count_vae):
    batch_cluster, latent, kl_divergence = model_vae.Encode(batch)
    loss_diffusion, latent_hat = caculate_diffusion_loss(args, model_reverse, diffusion, latent)
    batch_cluster_hat = model_vae.Decode(latent_hat)
    loss_vae = caculate_vae_loss(args, batch_cluster, batch_cluster_hat, kl_divergence, update_count_vae)
    
    # 해당 부분은 나중에 더 자세히 봐야겠다.
    # SNR과 관련 있나...?
    if args.anneal_steps > 0: 
        # anneal 한다.
        # 훈련이 진행될수록, diffusion 반영 정도가 줄어든다.
        # 최대는 args.lmbda이고 최소는 args.anneal_cap이다.
        anneal = max((1. - update_count / args.anneal_steps) * args.lamda, args.anneal_cap)
    else:
        anneal = args.lamda
    
    loss = loss_diffusion * anneal + loss_vae # 여기도 원래 논문의 코드와 다르다. 나중에 더 자세히 보자.
    

    return loss


def caculate_vae_loss(args, batch_cluster, batch_cluster_hat, kl_divergence, update_count_vae):
    loss = -torch.mean(torch.sum(F.log_softmax(batch_cluster_hat, 1) * batch_cluster, -1))  # multinomial log likelihood in MultVAE
    if args.vae_anneal_steps > 0: 
        # anneal 한다.
        # 훈련이 진행될수록, KL 반영정도를 높여야 한다.
        # args.vae_anneal_cap은 KL 반영정도의 최대값이다.
        anneal = min(args.vae_anneal_cap, 1. * update_count_vae / args.vae_anneal_steps)
    else:
        # anneal 안 한다.
        anneal = args.vae_anneal_ap
    
    return loss + anneal * kl_divergence


def caculate_diffusion_loss(args, model, diffusion, batch):
    """
    Importance sampling을 사용하기 위해 nn.MSE로 loss를 계산하지 않는다.
    Batch별 즉, user별 loss를 계산한 다음 importance sampling을 위해 Lt_history에 저장한다.
    그 다음, batch별 평균 값을 loss로 활용한다.
    """

    timesteps, pt = diffusion.sample_steps(batch.shape[0])
    if args.noise_scale != 0:
        x_t, noise = diffusion.get_noised_interaction(batch, timesteps)
    else:
        # Denoise 과정이 없다.
        # 즉, forward가 없다. 이러면 각 reverse는 x_t -> x_t를 복원하는 것이다.
        # 결국 x_t -> x_t를 하는 AutoEndocer를 여러 개 쌓은 것과 같다.
        x_t = batch


    batch_hat = model(x_t, timesteps)

    loss_batch_item = (batch_hat - batch) ** 2
    loss_batch = loss_batch_item.mean(dim=1)

    if args.snr is True:
        # timestep마다 loss weight를 다르게 둔다.
        weight = SNR(diffusion, timesteps - 1) - SNR(diffusion, timesteps)
        weight = torch.where((timesteps == 0), 1.0, weight)
    else:
        weight = torch.tensor([1.0] * batch.shape[0]).to(args.device)
    
    weighted_loss_batch = weight * loss_batch

    # Update Lt_history & Lt_count for importance sampling
    for timestep, loss in zip(timesteps, weighted_loss_batch):
        # loss는 timestep에 해당하는 loss 값이다.
        if diffusion.Lt_count[timestep] == diffusion.num_for_expectation:
            # 만약 history가 꽉 찼으면 old한 것을 버리고 새 것으로 채운다.
            Lt_history_old = diffusion.Lt_history.clone()
            diffusion.Lt_history[timestep, :-1] = Lt_history_old[timestep, 1:]
            diffusion.Lt_history[timestep, -1] = loss.detach()
        else:
            try:
                diffusion.Lt_history[timestep, diffusion.Lt_count[timestep]] = loss.detach()
                diffusion.Lt_count[timestep] += 1
            except:
                print(timestep)
                print(diffusion.Lt_count[timestep])
                print(loss)
                raise ValueError
    
    weighted_loss_batch /= pt # 논문 수식 14 참고.
    weighted_loss = weighted_loss_batch.mean()
    
    # weighted_loss : back propa에 사용
    # batch_hat: decoder input으로 사용.
    return weighted_loss, batch_hat 


def train_one_epoch(args, model_reverse, model_vae, diffusion, \
        optimizer_reverse, optimizer_vae, dataloader, update_count, update_count_vae):
    total_loss = 0.0
    for batch in dataloader:
        batch = batch.to(args.device)

        loss = caculate_loss(args, \
            model_reverse, model_vae, diffusion, batch, update_count, update_count_vae)

        total_loss += loss

        optimizer_reverse.zero_grad()
        optimizer_vae.zero_grad()
        loss.backward()
        optimizer_reverse.step()
        optimizer_vae.step()
        update_count_vae += 1


    return total_loss / len(dataloader), update_count_vae


def compute_metric(target_items, predict_items, topK):
    precisions = []
    recalls = []
    ndcgs = []
    mrrs = []
    num_users = len(predict_items)

    for k in topK:
        sum_precision = sum_recall = sum_ndcg = sum_mrr = 0.0
        for user_id in range(num_users):
            if len(target_items[user_id]) == 0: continue
            mrr_flag = True
            num_hit = user_mrr = dcg = 0
            
            for rank_idx in range(k):
                if predict_items[user_id][rank_idx] in target_items[user_id]:
                    num_hit += 1 # precision, recall에 사용
                    dcg += 1.0 / np.log2(rank_idx + 2)                    
                    if mrr_flag:
                        user_mrr = 1.0 / (rank_idx+1.0)
                        mrr_flag = False
            
            idcg = 0.0
            for rank_idx in range(len(target_items[user_id])):
                idcg += 1.0/np.log2(rank_idx+2)
            ndcg = (dcg/idcg)

            sum_precision += num_hit / k
            sum_recall += num_hit / len(target_items[user_id])
            sum_ndcg += ndcg
            sum_mrr += user_mrr

        precision = round(sum_precision / num_users, 4)
        recall = round(sum_recall / num_users, 4)
        ndcg = round(sum_ndcg / num_users, 4)
        mrr = round(sum_mrr / num_users, 4)

        precisions.append(precision)
        recalls.append(recall)
        ndcgs.append(ndcg)
        mrrs.append(mrr)

    return precisions, recalls, ndcgs, mrrs


def evaluate(args, model_reverse, model_vae, diffusion, loader, \
    label_items: sp.csr_matrix, consumed_items_mapped: sp.csr_matrix, topK: list):
    """
    Args
        args                    : hyper-parameters
        model_reverse           : 학습된 reverse diffusion model
        model_vae               : 학습된 MultVAE model
        diffsuion               : Diffusion
        loader                  : Test data loader // no_shffule
        label_items             : Ground Truth, shape: (num_users, num_items) 중에서 target item에만 1
        consumed_items          : training data에서 사용된 이미 user가 선호도를 보인 items. 나눠진 cluster 대로 mapping되어 있다.
        topK                    : top K list ex) [10, 20, 50]
    """
    model_reverse.eval()
    model_vae.eval()
    num_user = label_items.shape[0]
    user_idx_list = list(range(label_items.shape[0]))
    # target_items.shape[0] 대신 consumed_items.shape[0]도 ㄱㅊ

    predict_items = []
    target_items = []

    for user_id in range(num_user):
        # user_id에 해당하는, sp.csr_matrix로 저장되어 있는 user의 label item id를 list로 저장.
        # nonzero()하면 (row array, col array) 반환.
        # col array: np.ndarray의 idx 값이 item id임.
        target_items.append(label_items[user_id,:].nonzero()[1].tolist())

    with torch.no_grad():
        for batch_idx, batch in enumerate(loader):
            start_batch_user_id = batch_idx * args.batch_size
            end_batch_user_id = start_batch_user_id + len(batch)
            batch_consumed_items_mapped = consumed_items_mapped[user_idx_list[start_batch_user_id:end_batch_user_id]]
            batch = batch.to(args.device)
            _, batch_latent, _ = model_vae.Encode(batch)
            batch_latent_hat = diffusion.sample_new_interaction(model_reverse, batch_latent, steps=args.sampling_steps, sampling_noise=False)
            prediction_mapped = model_vae.Decode(batch_latent_hat)
            prediction_mapped[batch_consumed_items_mapped.nonzero()] = -np.inf

            _, indices_mapped = torch.topk(prediction_mapped, topK[-1]) # shape (batch[1].shape, topK[-1])
            indices = model_vae.cluster_map[indices_mapped]
            indices = indices.detach().cpu().numpy().tolist()
            predict_items.extend(indices)

        precisions, recalls, ndcgs, mrrs = compute_metric(target_items, predict_items, topK)
    
    return precisions, recalls, ndcgs, mrrs


def change_cols_accrd_to_map(sp_matrix, col_order_list, num_users, num_items):
    reverse_map = {col_order_list[i]:i for i in range(len(col_order_list))}
    id_users, id_items = sp_matrix.nonzero() # non_zero 값을 가지는 (rows), (cols) pair 반환.
    num_interactions = len(id_users)
    id_items_mapped = np.array([reverse_map[id_items[idx]] for idx in range(num_interactions)])

    data = np.ones(shape=(num_interactions,))
    row = id_users
    col = id_items_mapped
    sp_matrix_mapped = sp.csr_matrix(
        (data, (row, col)), dtype='float32', shape=(num_users, num_items)
    )

    return sp_matrix_mapped


def print_metric_results(topK, results):
    metric_list = ['Precision', 'Recall', 'nDCG', 'MRR']
    for idx, metric in enumerate(metric_list):
        str_result = ''
        for k_idx, k in enumerate(topK):
            str_metric = f'{metric}@{k}'
            str_result += f'    {str_metric:14s}: {results[idx][k_idx]:.4f}'
        print(str_result)


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

#### 3.2. main

In [8]:
dict_args = {}
args = dotdict(dict_args)

# Training hyper
args.dataset_name = 'ml-1m_clean'
args.file_name = 'ratings.dat'
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.batch_size = 400
args.lr = 1e-4
args.weight_decay = 0.0
args.epochs = 1000
args.topK = [10, 20, 50, 100]
args.num_cluster = 3
# diffusion의 denosing matching term도 KL_D를 줄이는 loss이다.
# 그렇기 때문에 VAE_loss는 recon error, diffsuion loss는 KL_D로 볼 수 있고,
# MultVAE처럼 VAE_loss와 Diffusion loss를 annealing할 수 있다.
args.anneal_cap = 0.005 
args.anneal_steps = 500
args.lamda = 0.3

# reverse hyper
args.hidden_dims_diffusion = [300]
args.norm = True

# vae hyper
args.hidden_dims_vae = [300]
args.num_clusters = 3
args.prob = 0.03 # for multinomial log-likelihood
args.vae_anneal_cap = 0.3
args.vae_anneal_steps = 200

# diffusion hyper
args.beta_start = 1e-4
args.beta_end = 0.02
args.noise_scale = 0.1
args.steps = 100
args.snr = True # assign different weight to different timestep or not
args.sampling_steps = 10



dir_path = os.path.join(os.getcwd(), args.dataset_name)
sp_train, sp_valid, sp_test, num_users, num_items, emb_items = data_load(dir_path)
train_dataset = DataDiffusion(torch.FloatTensor(sp_train.toarray()))
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, pin_memory=True, shuffle=True)
test_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)

# Build Diffusion and ReverseDiffusion
diffusion = Diffusion()
latent_dim = args.hidden_dims_vae[-1]
model_reverse = ReverseDiffusion(args.hidden_dims_diffusion, latent_dim).to(args.device)
optimizer_reverse = torch.optim.AdamW(model_reverse.parameters(), lr=args.lr, weight_decay=args.weight_decay)

# Build VAE
model_vae = VAE(num_items=num_items, emb_items=emb_items, \
    num_clusters=args.num_clusters, hidden_dims=args.hidden_dims_vae, device=args.device).to(args.device)
optimizer_vae = torch.optim.AdamW(model_vae.parameters(), lr=args.lr, weight_decay=args.weight_decay)

if args.num_clusters > 1:
    # VAE model에서 생성한 cluster map에 따라 sp_train의 item 순서를 바꾼다.
    sp_train_mapped = change_cols_accrd_to_map(sp_train, model_vae.cluster_map.detach().cpu().numpy(), num_users, num_items)
else:
    sp_train_mapped = sp_train

best_recall, best_epoch = -100, 0
best_test_result = None
print("Start training")
update_count_vae = 0
for epoch in range(args.epochs):
    if epoch - best_epoch >= 20: # early stopping
        print('-'*18)
        print('Exiting from training early')
        break

    model_reverse.train()
    model_vae.train()  
    start = time.time()
    avg_loss, update_count_vae = train_one_epoch(args, model_reverse, model_vae, diffusion, \
        optimizer_reverse, optimizer_vae, train_loader, epoch, update_count_vae)
    print(f'Epoch {epoch+1} -  train loss: {avg_loss: >10.4f},  time: {str(timedelta(seconds=int(time.time() - start)))}')

    if (epoch+1) % 5 == 0:
        val_results = evaluate(
            args, model_reverse, model_vae, diffusion, test_loader, sp_valid, sp_train_mapped, args.topK)
        test_results = evaluate(
            args, model_reverse, model_vae, diffusion, test_loader, sp_test, sp_train_mapped, args.topK)
    
        val_recalls = val_results[1]
        if val_recalls[1] > best_recall:
            best_recall, best_epoch = val_recalls[1], epoch
            print('  Update Best')

        print('  Validation data')
        print_metric_results(args.topK, val_results)
        print('  Test data')
        print_metric_results(args.topK, test_results)

user num: 5949
item num: 2810
running k-means on cuda..


[running kmeans]: 10it [00:00, 281.61it/s, center_shift=0.000061, iteration=10, tol=0.000100]

Start training


Epoch 1 -  train loss:   992.6836,  time: 0:00:02
Epoch 2 -  train loss:   995.6473,  time: 0:00:01
Epoch 3 -  train loss:   990.9977,  time: 0:00:01
Epoch 4 -  train loss:  1026.1639,  time: 0:00:02
Epoch 5 -  train loss:   953.3059,  time: 0:00:02
  Update Best
  Validation data
    Precision@10  : 0.0217    Precision@20  : 0.0179    Precision@50  : 0.0143    Precision@100 : 0.0120
    Recall@10     : 0.0160    Recall@20     : 0.0238    Recall@50     : 0.0439    Recall@100    : 0.0688
    nDCG@10       : 0.0152    nDCG@20       : 0.0200    nDCG@50       : 0.0294    nDCG@100      : 0.0395
    MRR@10        : 0.0568    MRR@20        : 0.0631    MRR@50        : 0.0682    MRR@100       : 0.0700
  Test data
    Precision@10  : 0.0129    Precision@20  : 0.0101    Precision@50  : 0.0081    Precision@100 : 0.0067
    Recall@10     : 0.0202    Recall@20     : 0.0282    Recall@50     : 0.0511    Recall@100    : 0.0783
    nDCG@10       : 0.0144    nDCG@20       : 0.0182    nDCG@50       : 0.02